# DSCI310_Group8 Project Milestone1

In [133]:
library(tidyverse)
library(digest)
library(repr)
library(tidymodels)
library(readxl)
library(cowplot)
library(GGally)
library(broom)
library(rlang)
library(testthat)
options(repr.matrix.max.rows = 6)

set.seed(123)
source("../src/r/load_data.r")
source("../src/r/wrangle_data.r")
source("../src/r/summary_fun.r")
source("../src/r/num_na.r")


In [134]:
dest <- "../data/raw/student_performance.xls"
cols <- c("STG", "SCG", "STR", "LPR", "PEG", "UNS")
data_1 <- load_data(dest, 2, STG, SCG, STR, LPR, PEG, UNS)
data_2 <- load_data(dest, 3, STG, SCG, STR, LPR, PEG, UNS)



ERROR: Error in load_data(dest, 2, STG, SCG, STR, LPR, PEG, UNS): object 'STG' not found


In [93]:
#user_training <- data_1 %>%
#select(STG,PEG) 
#interest_variables <- c("STG", "PEG")
user_training <- wrangle_data(data_1)

In [132]:
#user_testing <-data_2 %>%
#select(STG,PEG)
user_testing <- wrangle_data(data_2)
head(user_testing)

STG,PEG
<dbl>,<dbl>
0.00,0.05
0.05,0.14
0.08,0.85
0.20,0.85
0.22,0.90
0.14,0.30


In [112]:
#user_means <- user_training %>%
#map_df(mean, na.rm =TRUE)
user_means <- summary_fun(user_training,mean)
user_means

STG,PEG
<dbl>,<dbl>
0.3711473,0.4585388


In [113]:
#maximum <- user_training %>%
#map_df(max,na.rm =TRUE)

maximum <- summary_fun(user_training, max)
maximum

STG,PEG
<dbl>,<dbl>
0.99,0.93


Table 1.7.2 The maximums of STG and PEG

The following code is used to find the minimums of the predictor and explanatory variables (STG and PEG) we plan to use in analysis.

ERROR: Error in eval(expr, envir, enclos): object 'va' not found


In [72]:
#minimum <- user_training %>%
#map_df(min,na.rm =TRUE)

minimum <- summary_fun(user_training, min)
minimum

STG,PEG
<dbl>,<dbl>
0,0


Table 1.7.3 The minimums of STG and PEG

In [101]:
observations <- user_training %>%
summarize(n = n())
observations

n
<int>
258


In [127]:
source("../src/num_na.r")

In [128]:
#NA_STG_train<-sum(is.na(user_training$STG))

NA_STG_train <- num_na(user_training, STG)
NA_STG_train

NA_PEG_train <- num_na(user_training, PEG)
NA_PEG_train

NA_STG_test <- num_na(user_testing, STG)
NA_STG_test

NA_PEG_test <- num_na(user_testing, PEG)
NA_PEG_test

[1] 0

[1] 0

[1] 0

[1] 0

In [ ]:
peg_stg <- user_training %>%
    ggplot(aes(x=STG, y=PEG))+ 
    geom_point()+
    labs(y = "Exam performance of user for goal objects (PEG)" , x = "Degree of study time for goal object materials (STG)", title = "PEG vs. STG")+
    theme(text=element_text(size=15))
peg_stg

#### 2.2 Linear Regression

Creating and assigning our linear regression model specification to an object called lm_spec.


In [ ]:
lm_spec <- linear_reg() %>%
        set_engine("lm") %>%
        set_mode("regression")

Creating a recipe for the model and assigning it to credit_recipe.

In [ ]:
performance_recipe <- recipe(PEG~.,data=user_training)

Fitting our simple linear regression model by using the model specification and recipe.

In [ ]:
performance_fit <- workflow() %>%
add_recipe(performance_recipe) %>%
add_model(lm_spec) %>%
fit(data = user_training)

performance_fit

Table 2.2.1 the model of linear regression between the STG and PEG

Looking at the slopes and coefficients from the table 2.2.1 from each of the predictors, we can create our prediction model:


Linear Regression Model: the degree of student performance = 0.3656 + 0.2505 * the degree of study time

As the degree of study time increases in one unit, the degree of student performance will increase by 0.2505 units.

We haven't declared the units of STG and PEG here since both STG and PEG are expressed in degree in the oriignal dataset.

Calculating the RMSE to assess goodness of fit on performance_fit

In [ ]:
lm_rmse <- performance_fit %>%
        predict(user_training) %>%
        bind_cols(user_training) %>%
        metrics(truth = PEG ,estimate = .pred) %>%
        filter(.metric == "rmse") %>%
        select(.estimate) 
lm_rmse

knn_rmspe <- performance_fit_knn %>%
  predict(user_testing) %>%
  bind_cols(user_testing) %>%
  metrics(truth = PEG, estimate = .pred)%>%
  filter(.metric == 'rmse') %>%
  pull(.estimate) 

knn_rmspeTable 2.2.1 The RMSE (of our training data) of linear regression

Calculating the RMSPE to assess how well the model predicts on the testing data

In [ ]:
lm_rmspe <- performance_fit %>%
        predict(user_testing) %>%
        bind_cols(user_testing) %>%
        metrics(truth = PEG ,estimate = .pred) %>%
        filter(.metric == "rmse") %>%
        select(.estimate) 
lm_rmspe

Table 2.2.2 The RMSPE (of our testing data) of linear regression

The RMSE (of our training data) of linear regression is 0.249.

The RMSPE (of our testing data) of the linear regression is 0.281.

#### 2.3 K-NN Regression

Because we have already set the seed at the beginning of the report, we don't need to set the seed again. The procedure is reproducible.

Scaling and shifting our predictors on the training data

Creating and assigning the K-NN regression model specification to an object called performance_knn_recipe.

In [ ]:
performance_knn_recipe <- recipe(PEG ~., data = user_training) %>%
                        step_center(all_predictors()) %>%
                        step_scale(all_predictors())

performance_knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
                      set_engine("kknn") %>%
                      set_mode("regression")

performance_vfold <- vfold_cv(user_training, v = 5, strata = PEG)

performance_knn_workflow <- workflow() %>%
                        add_recipe(performance_knn_recipe) %>%
                        add_model(performance_knn_spec)

gridvals <- tibble(neighbors = seq(1,20))

performance_knn_results <- performance_knn_workflow %>%
                       tune_grid(resamples = performance_vfold, grid = gridvals) %>%
                       collect_metrics() 


Selecting the value of k resulting in best RMSE

In [ ]:
kmin <- performance_knn_results %>%
               filter(.metric == 'rmse') %>%
               filter(mean == min(mean))  %>% 
               pull(neighbors)
kmin

The best value of K (and hence the one we will be using) yielding the lowest error is K = 20.

Retraining the model using K = 20 and predicting on held-out (testing) data

In [ ]:

performance_spec_knn <- nearest_neighbor(weight_func = "rectangular", neighbors = kmin) %>%
  set_engine("kknn") %>%
  set_mode("regression")

performance_fit_knn <- workflow() %>%
  add_recipe(performance_knn_recipe) %>%
  add_model(performance_spec_knn) %>%
  fit(data = user_training)

knn_rmse <- performance_fit_knn %>%
  predict(user_training) %>%
  bind_cols(user_training) %>%
  metrics(truth = PEG, estimate = .pred)%>%
  filter(.metric == 'rmse') 

knn_rmse

Table 2.3.1 The RMSE (of training data) of K-NN regression.

In [ ]:
knn_rmspe <- performance_fit_knn %>%
  predict(user_testing) %>%
  bind_cols(user_testing) %>%
  metrics(truth = PEG, estimate = .pred)%>%
  filter(.metric == 'rmse') 

knn_rmspe

Table 2.3.2 The RMSPE (of testing data) of K-NN regression.

The RMSE (of training data) of K-NN regression is 0.227.

The RMSPE (of testing data) of K-NN regression is 0.257.

#### 2.4 Visualization of Linear Regression

In [ ]:
lm_predictions <- user_training %>%
    ggplot(aes(x = STG, y = PEG)) +
        geom_point() +
        geom_smooth(method = "lm", se = FALSE) +
        xlab("Degree of study time") +
        ylab("Degree of student performance") +
        ggtitle("Linear regression fitting model between STG and PEG")+
        theme(text = element_text(size = 14))
lm_predictions

Figure 2.4.1 Linear regression fitted over STG and PEG

Figure 2.4.1 shows our linear regression model overlaid our testing data with STG on the x-axis and PEG on the y-axis. 

We can see that since the data itself is quite spread out, the plotted line of our linear regression cuts through the points, as linear regression usually does. We can see that the points are distributed relatively evenly over and under our fitted line. 

#### 2.5 Visualization of the K-NN Regression

In [ ]:
knn_pred <- performance_fit_knn %>%
  predict(user_training) %>%
  bind_cols(user_training)
knn_regression_plot <- ggplot(user_training, aes(x = STG, y = PEG)) +
  geom_point(alpha = 0.4) +
  geom_line(data = knn_pred, 
            mapping = aes(x = STG, y = .pred), 
            color = "blue") +
  xlab("Degree of study time") +
  ylab("Degree of study performance") +
  ggtitle("K-NN regression fitting model between STG and PEG") +
  theme(text = element_text(size = 14))

knn_regression_plot

Figure 2.5.1 the K-NN regression between STG and PEG

Figure 2.5.1 shows our K-NN regression model overlaid our testing data with STG on the x-axis and PEG on the y-axis. 

We can see that although the data itself is quite spread out, the plotted line of our K-NN regression tries to follow the data points by producing “wiggles”. 
This indicates that it is more flexible than our linear model, since it tries to follow most of the points instead of cutting through them.

However, it is unclear if our model underfits or overfits the data since the testing data points themselves are spread out across the entire graph and there is a lot of randomness in our data.

#### 2.6 Comparing and Contrasting End Results

Looking at the visualizations of both methods, we cannot evidently see which model is better. The linear regression model has a relatively even ratio of points above and under the fitted line, but our K-NN Regression is also flexible and follows the data points. Hence, we must rely on the calculated RMSPE to determine the better model.

As seen in Sections 2.2 and 2.3, the the RMSE (of our training data) of linear regression is 0.249, and the RMSPE (of our testing data) of the linear regression is 0.281. The RMSE (of training data) of K-NN regression is 0.227, and the RMSPE (of testing data) of K-NN linear regression is 0.257. By comparing the RMSE of both methods, we can see that our K-NN regression model has a slightly lower RMSE on our training data. By comparing the RMSPE of both methods, we can see that again our K-NN regression model has a slightly lower RMSPE on our testing data. Therefore, our K-NN Regression Model is slightly more accurate.

Due to the higher accuracy of our K-NN Regression, our final visualization of the best model is shown in Figure 2.5.1. 

### 3. Discussion

Before performing the analysis, we had expected to find a positive, linear relationship between PEG and STG. As a student spends more time studying, they should perform better. As a result of our analysis, we have found the accuracy for our linear regression model to be 0.257, while the accuracy of our K-NN regression is 0.227 (where the range of degree is 1). 

Both of these types of regression have a prediction error percentage of about 40% (therefore our accuracy is about 60%), but as discussed in 2.6, our K-NN Regression model is slightly better than our linear regression due to its higher accuracy, which is consistent with our guess in Methods. Our model has relatively low accuracy. 

This was not quite what we had expected to find as the outcome of our analysis, since we initially expected to be able to observe a clearer positive relationship between STG and PEG. However, this can still be reasoned with the fact that there was already not much of a strong relationship between STG and PEG as described in Section 1.8.  

As mentioned in the Introduction, we had initially believed the accuracy of our K-NN regression to be about 80%. Since there are no missing observations in our data set, there may be several reasons causing this difference in estimated accuracy and the actual accuracy calculated. On one hand, this can be attributed to the fact that exam performance could be affected by other external factors such as health condition, student IQ, stress levels, learning ability, etc. that were not in the data set. On the other hand, we can also say that our data set may not be big enough to directly draw a relationship between just study time and exam performance. 

It would be best to seek out another form of prediction that can predict using multiple factors (which would be outside the scope of what we have learned thus far in our course), or find a larger data set with more observations.

In conclusion, our findings will somewhat help students assess how well they are performing in comparison to the time they are spending studying. As you spend more time studying, you may perform better, but other external factors can also come into play and affect your final results. Since it is final season, it is important to remember that although studying is extremely important, it is also crucial to take care of other things such as your physical and mental health. 

This study can also be beneficial to find the best study methods and learn how to study more efficiently. Some future questions to consider:

* How much time should a student be spending studying to improve exam performance?
* How do other factors (such as repetition, knowledge level, etc.) contribute to exam performance?
* Can a similar approach be used in the industry to predict workers' performance based on their working time? How?

### 4. References

Colak, I., Sagiroglu, S., & Kahraman, H. T. (2008, December). A user modeling approach to web based adaptive educational hypermedia systems. In 2008 Seventh International Conference on Machine Learning and Applications (pp. 694-699). IEEE.

H. T. Kahraman, Sagiroglu, S., Colak, I., Developing intuitive knowledge classifier and modeling of users' domain dependent data in web, Knowledge Based Systems, vol. 37, pp. 283-295, 2013.

Kahraman, H. T. (2009). Designing and Application of Web-Based Adaptive Intelligent Education System. Gazi University Ph. D. Thesis, Turkey, 1-156.

Tsiriga, V., & Virvou, M. (2004). A framework for the initialization of student models in web-based intelligent tutoring systems. User Modeling and User-Adapted Interaction, 14(4), 289-316.

UCI Machine Learning Repository: User Knowledge Modeling Data Set. (2009, October). Machine Learning Repository. Retrieved Novermber 6, 2021, from https://archive.ics.uci.edu/ml/datasets/User+Knowledge+Modeling 